# Karpathy Lecture 4 - backpropogation
This lecture focuses on implementing backward pass (gradients) from scratch. We will define our gradients and then compare them with pytorch's autograd values.

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import random

import matplotlib.pyplot as plt # for making figures
%matplotlib inline

# Utility

In [3]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

# Data

In [4]:
def prepare_dataset(words, block_size=3):
    """Takes a list of words and block size and prepares dataset for NN training"""

    # build dataset
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        word = w + "."

        for ch in word:
            ch_idx = stoi[ch]
            X.append(context)
            Y.append(ch_idx)

            # update context
            new_context = context[1:] + [ch_idx]
            context = new_context
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

# intake raw data
path = "/Users/andylee/Projects/ml-tutorials/data/names.txt"
words = open(path, 'r').read().splitlines()

# build lookup tables
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

# split dataset
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

block_size = 3
vocab_size = len(stoi)

Xtr,  Ytr  = prepare_dataset(words[:n1], block_size=block_size)     # 80%
Xdev, Ydev = prepare_dataset(words[n1:n2], block_size=block_size)   # 10%
Xte,  Yte  = prepare_dataset(words[n2:], block_size=block_size)     # 10%

# Build network
This section implements a simple MLP with batchnorm 

In [5]:
# set seed
g = torch.Generator().manual_seed(2147483647) # for reproducibility

# hyperparameters
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

# initialization parameters
kaiming_initialization = (5/3)/((n_embd * block_size)**0.5)
epsilon_initialization = 0.01

# network - embedding
C  = torch.randn((vocab_size, n_embd),            generator=g)

# network - mlp
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * kaiming_initialization
b1 = torch.randn(n_hidden,                        generator=g) * epsilon_initialization
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * epsilon_initialization
b2 = torch.randn(vocab_size,                      generator=g) * epsilon_initialization

# network - batchnorm
batchnorm_gain = torch.ones((1, n_hidden))
batchnorm_bias = torch.zeros((1, n_hidden))
batchnorm_mean_running = torch.zeros((1, n_hidden))
batchnorm_var_running = torch.ones((1, n_hidden))

# network parameters to optimize
parameters = [C, W1, W2, b2, batchnorm_gain, batchnorm_bias]
for p in parameters:
  p.requires_grad = True

total_parameters = sum(p.nelement() for p in parameters) # number of parameters in total
print(f" Total network parameters: {total_parameters}")

 Total network parameters: 12097


## Optimization Loop
This section builds the optimization loop for the network

In [16]:
# same optimization as last time
max_steps = 20000
batch_size = 2
n = batch_size # for shorter 
vocab_size = vocab_size

lossi = []
for i in range(max_steps):
    
    # generate mini-batch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y

    # forward pass - embedding layer
    emb = C[Xb]  # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1)  # concatenate the vectors
    
    # forward pass - linear layer
    linear_out = embcat @ W1 + b1

    # forward pass - batchnorm
    batch_mean = (1/n) * linear_out.sum(0, keepdim=True)
    batch_variance = 1/(n-1) * ((linear_out - batch_mean)**2).sum(0, keepdim=True) 
    batch_std_inverse = (batch_variance + 1e-5)**-0.5
    batchnorm_out = (linear_out - batch_mean) * batch_std_inverse
    batchnorm_out = batchnorm_gain * batchnorm_out + batchnorm_bias
    
    # log running means & variances
    with torch.no_grad():
        bnmean_running = 0.999 * batchnorm_mean_running + 0.001 * batch_mean
        bnstd_running = 0.999 * batchnorm_var_running + 0.001 * batch_variance

    # forward pass - non-linear
    hidden_out = torch.tanh(batchnorm_out)

    # forward pass - softmax 
    logits = hidden_out @ W2 + b2  # output layer
    logits_maxes = logits.max(1, keepdim=True).values # numerical stability hack
    logits_normalized = logits - logits_maxes

    # forward pass - softmax
    counts = torch.exp(logits_normalized)
    counts_sum = counts.sum(1, keepdim=True) # output (m, 1)
    counts_sum_inverse = counts_sum ** -1
    probabilities = counts * counts_sum_inverse

    # forward pass - log probabilities
    log_probabilities = probabilities.log()

    # forward pass - compute loss
    # E.g. 0th batch row, Yb=5 (e.g. E), then you take neg log probabilty of E and calculate loss
    cross_entropy_loss = -log_probabilities[range(n), Yb].mean(0, keepdim=True)

    # capture intermediate gradients for inspection
    for t in [emb, 
              batchnorm_out, 
              logits,
              logits_maxes,
              logits_normalized,
              counts,
              counts_sum,
              counts_sum_inverse,
              probabilities,
              log_probabilities]:

        t.retain_grad()

    # backward pass 
    cross_entropy_loss.backward()
    break

In [17]:
# backward pass - manual gradient calculations

# intermediaries
Yb_one_hot = F.one_hot(Yb, num_classes=vocab_size)

# gradient calculations
dlog_probabilities = -1.0/batch_size * Yb_one_hot


log_probabilities | exact: True  | approximate: True  | maxdiff: 0.0


### Compute gradients
This section manually calculates gradients for backward pass

In [14]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

cmp('logprobs', dlog_probabilities, log_probabilities)
# cmp('probs', dprobs, probs)
# cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
# cmp('counts_sum', dcounts_sum, counts_sum)
# cmp('counts', dcounts, counts)
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs        | exact: False | approximate: False | maxdiff: 25.15879249572754


## Pytorchify code
This section converts raw network into modules that look like pytorch.

In [92]:
class Linear: 
    def __init__(self, fan_in, fan_out, bias=True):
        self.bias = bias 
        self.weight = torch.randn(fan_in, fan_out, generator=g) / fan_in ** 0.5
        self.b = torch.zeros(fan_out) if bias else None
    
    def __call__(self, x):
        if self.bias: 
            self.out = x @ self.weight + self.b
        else:
            self.out = x * self.weight
        
        return self.out

    def parameters(self):
        return [self.weight, self.b] if self.bias else [self.weight]

class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.dim = dim
        self.eps = eps
        self.momentum = momentum

        self.gamma = torch.randn(dim)
        self.beta = torch.randn(dim)

        self.running_mean = torch.zeros(dim)
        self.running_std = torch.ones(dim)

    def __call(self, x):
        if self.training:
            # calculate batch statistics
            batch_mean = x.mean(0, keepdim=True)
            batch_var = x.var(0, keepdim=True)

            with torch.no_grad():
                self.running_mean = (1-self.momentum) * self.running_mean + self.momentum * batch_mean
                self.running_var = (1-self.momentum) * self.running_var + self.momentum * batch_var

        else:
            batch_mean = self.running_mean
            batch_std = self.running_std
        
        xhat = (x - batch_mean) / torch.sqrt(batch_var + self.eps) ** 0.5
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]


class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []

### Build pytorch network
This section builds the network using pytorch layers defined above. 

In [97]:
# setup NN parameters

# generator
g = torch.Generator().manual_seed(2147483647) # for reproducibility

# NN parameters
n_vocab = vocab_size
n_embd = 10 
n_hidden = 100

# define pytorch layers
C = torch.randn(vocab_size, n_embd)
layers = [
    # stacked layers
    Linear(n_embd*block_size, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),

    # logits layer
    Linear(n_hidden, vocab_size)
]

with torch.no_grad():
    # make last layer less confident
    layers[-1].weight *= 0.1 

    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 5/3  # Ensure layer.weight modification correctly applies only to tensors
    
parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True


46497


### Setup optimization loop

In [14]:
# Correct optimization loop
max_steps = 20000
batch_size = 32
lossi = []

for i in range(max_steps):
  
    # generate mini-batch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

    # embed input
    emb = C[Xb] # embed input
    x = emb.view(emb.shape[0], -1) # reshape dimensions

    # forward pass
    for layer in layers:
        x = layer(x)

    # loss calculation
    loss = F.cross_entropy(x, Yb)

    # backward pass
    for layer in layers:
        if hasattr(layer, 'out'):
            layer.out.retain_grad()
    
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update parameters
    learning_rate = 0.1 if i < 100000 else 0.01 
    for p in parameters:
        p.data += -learning_rate * p.grad
      
    # track stats
    if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

    if i >= 1000:
        break # AFTER_DEBUG: would take out obviously to run full optimization

NameError: name 'layers' is not defined

# Visualizations

These are diagnostic tools that Karpathy has introduced us for understanding whether or not the network is updating properly based on our optimization parameters and initialization of weights. It's a way to quickly introspect the network and find if there are things that are wrong. 

In [13]:
plt.figure(figsize=(20, 4))
legends = []

for i, layer in enumerate(layers[:-1]):
    if isinstance(layer, Tanh):
        t = layer.out
        print('layer %d (%10s): mean %+.2f, std %.2f, saturated: %.2f%%' % (i, layer.__class__.__name__, t.mean(), t.std(), (t.abs() > 0.97).float().mean()*100))
        
        hy, hx = torch.histogram(t, density=True)
        plt.plot(hx[:-1].detach(), hy.detach())
        legends.append(f'layer {i} ({layer.__class__.__name__}')

plt.legend(legends);
plt.title('activation distribution')

NameError: name 'layers' is not defined

<Figure size 2000x400 with 0 Axes>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0129c86c-0221-47a9-9ff5-5a790f5a7462' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>